In [ ]:
using FFTW
using Plots

In [ ]:
L = 10
N = 300
dx = L/N
dk = 2π/L
dt = 1e-2

In [ ]:
x1 = dx .* collect(0:N-1)
x2 = dx .* collect(0:N-1)'
k1 = rfftfreq(N , N * dk)
k2 = fftfreq(N, N * dk)';

In [ ]:
k² = @. (k1^2 + k2^2);
k = sqrt.(k²)
c = - k²
f1 = exp.(c .*dt)
f2 = (f1 .- 1.) ./ c
f2[1, 1] = 0

In [ ]:
F = plan_rfft(x1 .+ x2)
B = plan_irfft(im*k², N);

In [ ]:
kmax = maximum(k)
twothirds = k .> kmax * 2/3
function antialias!( F )
    F[twothirds] .= 0.
end

In [ ]:
V = @. cos(dk * x1) * cos(dk * x2) / 5
FV = F*V
v = -10*(B*(im .* k .* FV))

function etd!(Fφ, φ)
    Fvφ = F*(v.*φ)
    @. Fφ = f1 * Fφ
    @. Fφ += - 2*im * (f2 * k * Fvφ)
    φ .= B*Fφ
    antialias!(Fφ)
end

In [ ]:
dφ = zeros((N, N))
φ = zeros((N, N))
φ[1:10,1:10] .= 1.
# φ = @. exp(- ((x1-L/2)^2 + (x2-L/2)^2) / 2)
Fφ = F*φ
plot(x1, x2, φ; st=:surface, zrange=(-.1, 1.1))

In [ ]:
plot(x1, x2, V; st=:surface)

In [ ]:
V

In [ ]:
M = 10
for t in 1:M
    etd!(Fφ, φ)
end
plot(; zrange=(-1, 1))
plot!(x1, x2, φ; st=:surface)

In [ ]:
frames = 1000

dφ = zeros((N, N))
φ = zeros((N, N))
φ[1:100, 1:100] .= 1.
Fφ = F*φ

anim = @animate for i in 1:frames
    if i>1
        for i in 2_000
            etd!(Fφ, φ)
        end
    end
    plot(x1, x2, φ; st=:surface, zrange=(-.1, 1.5))
    plot!(size=(1200, 1200))
end every 1;
gif(anim, "diff_2D.mp4", fps = 20)